In [1]:
import pandas as pd
import os
import numpy as np

import ast
import os
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio 

pio.renderers.default = "plotly_mimetype"

This notebook is created by: Tan Nguyen on 26-10-2023 to extract BOLD data from 1d files to csv files. These csv files will contain both timestamps and BOLD signal after preprocessing (e.g. xcp_24p_gsr). 

In [43]:
# ROOT directory and fixed variables
in_path = "/data/nil-external/dcl/Events152_fMRI_NeuralMechanisms/voxelwiseAnalyses/"
sub_ids = [f"0{i}" for i in range(1, 10)] + [str(x) for x in list(range(10, 48))]
# note that run_ids are 1-indexed because it's copied from R, subtract 1 when indexing when appropriate (e.g. mov_ids[r_id - 1] or nframes[r_id - 1])
run_ids = list(range(1, 5))
nframes = [405, 467, 404, 444]
mov_ids = ["1.2.3", "6.3.9", "3.1.3", "2.4.1"]

# Subject-specific parameters
onset_tbl = pd.read_csv(f"{in_path}subject_movie_onset/e152onsets.txt", delim_whitespace=True)

# Parcel-specific parameters
suff_sch_np2 = "np2_Sch400x7"
suff_sch_xcp = "schaefer400x7_ts"
suff_subcortical_np2 = "np2_subcortical"
suff_subcortical_xcp = "TianSubcortexS2x3T_ts"

preproc = "xcp_24p_gsr"

TR = 1.483

subcort_tbl = pd.read_csv(f"{in_path}atlas/indexTable.csv")
subcortical_ids = subcort_tbl['node_label'].tolist()

sch_tbl = pd.read_csv(f"{in_path}atlas/schaefer400x7NodeNames_updated.txt", header=None)
sch_ids = sch_tbl.iloc[0:400, 0].tolist()


In [44]:
def read_xcp_schaefer(sid, rid):
    fname = os.path.join(in_path, f"parcel_timeseries/{preproc}/XCP_OUTPUT_{sid}/sub-{sid}/run-{rid}/fcon/schaefer400x7/sub-{sid}_run-{rid}_{suff_sch_xcp}.1D")
    
    if os.path.exists(fname):
        xcp_tbl = pd.read_csv(fname, header=None, delim_whitespace=True)
        
        # Check if any column in xcp_tbl has all values equal to 0
        if any(xcp_tbl.apply(lambda x: all(x == 0), axis=0)):
            raise ValueError(f"all 0s in xcp.tbl for at least 1 parcel for subject {sid} run {rid}")
        
        # Check if the number of rows and columns match the expected values
        if xcp_tbl.shape[0] != nframes[rid-1] or xcp_tbl.shape[1] != len(sch_ids):
            raise ValueError(f"wrong xcp.tbl: {xcp_tbl.shape[0]} rows, {xcp_tbl.shape[1]} columns, expected {nframes[rid]} rows, {len(sch_ids)} columns")
        
        return xcp_tbl
    else:
        msg = f"missing {fname}"
        raise FileNotFoundError(msg)

def read_xcp_subcortical(sid, rid):
    fname = os.path.join(in_path, f"parcel_timeseries/{preproc}/XCP_OUTPUT_{sid}/sub-{sid}/run-{rid}/fcon/TianSubcortexS2x3T/sub-{sid}_run-{rid}_{suff_subcortical_xcp}.1D")
    
    if os.path.exists(fname):
        xcp_tbl = pd.read_csv(fname, header=None, delim_whitespace=True)
        
        # Check if any column in xcp_tbl has all values equal to 0
        if any(xcp_tbl.apply(lambda x: all(x == 0), axis=0)):
            raise ValueError(f"all 0s in xcp.tbl for at least 1 parcel for subject {sid} run {rid}")
        
        # Check if the number of rows and columns match the expected values
        if xcp_tbl.shape[0] != nframes[rid-1] or xcp_tbl.shape[1] != len(subcortical_ids):
            raise ValueError("wrong xcp.tbl")
        
        return xcp_tbl
    else:
        msg = f"missing {fname}"
        raise FileNotFoundError(msg)


In [48]:
# define a dictionary of dataframes, where the key is the subject ID and run ID and parcel type the value is a dataframe created by the two loops below
sub_run_parceltype = {}
# Assuming read_xcp_schaefer and read_xcp_subcortical, are defined
# Also assuming that sub_ids, sch_ids, and subcortical_ids are defined
for rid in run_ids:
    for do_par in ["Schaefer2018_400x7", "subcortical"]:
        p_ids = sch_ids if do_par == "Schaefer2018_400x7" else subcortical_ids
        for sid in sub_ids:
            parcel_tbl = read_xcp_schaefer(sid, rid) if do_par == "Schaefer2018_400x7" else read_xcp_subcortical(sid, rid)

            parcel_tbl = pd.DataFrame(parcel_tbl)
            parcel_tbl.columns = p_ids
            # each row in parcel_tbl here contain BOLD activity at a given TR (frame) for all parcels
            # fMRI frames were obtained before the movie started, and the difference between the first frame and the first onset for each subject is defined in onset_tbl
            # use the onset_tbl and TR to add timestamp in seconds for parcel_tbl
            # note that the first frame is indexed as 0, so the first frame is 0 * TR + onset_tbl["onset"][sid - 1]
            parcel_tbl["time_second"] = parcel_tbl.index * TR - onset_tbl.loc[onset_tbl["sub.id"] == f"sub-{sid}"][f"run{rid}_{mov_ids[rid-1]}"].values[0]
            sub_run_parceltype[f"sub-{sid}_{mov_ids[rid-1]}_{do_par}"] = parcel_tbl

In [52]:
sub_run_parceltype['sub-03_6.3.9_Schaefer2018_400x7']

,LH_Vis_1,LH_Vis_2,LH_Vis_3,LH_Vis_4,LH_Vis_5,LH_Vis_6,LH_Vis_7,LH_Vis_8,LH_Vis_9,LH_Vis_10,...,RH_Default_pCunPCC_1,RH_Default_pCunPCC_2,RH_Default_pCunPCC_3,RH_Default_pCunPCC_4,RH_Default_pCunPCC_5,RH_Default_pCunPCC_6,RH_Default_pCunPCC_7,RH_Default_pCunPCC_8,RH_Default_pCunPCC_9,time_second
0,-13.974860,-12.051970,-23.584250,-20.451580,-17.576560,-29.673340,-16.011430,-22.712210,-37.136840,2.815336,...,2.668356,8.884113,19.047450,4.783875,-0.223456,-1.230312,1.128965,13.324600,3.510117,-10.109722
1,-15.302380,-13.467100,-29.006840,-23.482130,-21.048020,-27.422030,-20.571610,-21.051900,-42.781310,-7.800775,...,0.797366,12.888190,20.691800,12.931610,1.941872,-1.448962,4.175281,23.844390,12.571060,-8.626722
2,-7.068161,-6.200527,-18.878080,-15.599410,-13.168720,-4.053083,-17.722590,0.080307,-27.546250,-14.719360,...,-0.700265,25.142490,21.024470,21.192740,10.475380,-4.052744,2.773781,29.131320,16.307500,-7.143722
3,4.095456,0.233543,-4.307902,-8.785852,-4.104116,12.089990,-10.983350,16.094970,-3.365862,-8.758202,...,-1.573616,29.690280,18.027060,22.544160,21.463370,-6.818582,-2.460805,25.392160,9.291926,-5.660722
4,-0.673357,-5.820334,-9.934291,-16.728070,-16.310590,-16.333880,-7.084727,5.027834,-19.762060,-4.953893,...,-6.736323,6.527455,3.719626,16.900280,21.521220,-4.554370,-2.214622,11.739700,-1.325770,-4.177722
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
462,5.644018,-0.687174,7.171595,5.667608,2.741308,9.020585,0.293150,-16.114950,-7.799092,13.067560,...,24.161140,9.946978,29.980600,7.568543,25.262580,12.279680,-6.830762,9.731444,25.616460,675.036278
463,5.218675,-0.892719,-15.658620,4.155960,-3.945707,-29.696860,0.102780,-54.046200,-38.846600,20.096410,...,13.005830,7.343487,19.440100,12.857340,13.296230,9.052876,-2.354718,26.124450,32.301440,676.519278
464,12.658950,8.370711,-20.455480,10.978880,6.821104,-32.593350,0.103995,-49.870120,-48.981470,23.507230,...,0.625955,11.688660,-7.294926,4.346623,-5.251982,-2.810031,2.004254,19.634630,16.338230,678.002278
465,4.422587,6.524461,-24.956490,9.647304,11.634400,-45.227460,-0.344420,-52.681810,-47.435270,16.469170,...,-9.990725,18.468280,-13.031190,1.373162,-13.481140,-9.382021,6.369887,16.658730,9.454695,679.485278


In [55]:
# for each type of parcel (schaefer and subcortical), create a directory to store csv files. File name contains subject ID, run ID, and parcel type
for do_par in ["Schaefer2018_400x7", "subcortical"]:
    if not os.path.exists(f"{in_path}parcel_timeseries/{preproc}_csv/{do_par}"):
        os.makedirs(f"{in_path}parcel_timeseries/{preproc}_csv/{do_par}")

    # for each subject and run, save the corresponding dataframe as a csv file
    for sid in sub_ids:
        for rid in run_ids:
            sub_run_parceltype[f"sub-{sid}_{mov_ids[rid-1]}_{do_par}"].to_csv(f"{in_path}parcel_timeseries/{preproc}_csv/{do_par}/sub-{sid}_run-{rid}_{mov_ids[rid-1]}_{do_par}.csv", index=False) 